In [105]:
# Import libraries

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model

In [18]:
import pandas as pd

# Load the dataset without explicit date parsing
file_path = 'justin.csv'
df = pd.read_csv(file_path)

# Preprocess 'Date' and 'Time' columns into a combined datetime column
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%m/%d %H:%M')

# Drop the original 'Date' and 'Time' columns if not needed
df.drop(['Date', 'Time'], axis=1, inplace=True)

# Display the first few rows of the dataset to verify
print(df.head())


    Location            DateTime
0     Stever 1900-06-30 23:09:00
1     Stever 1900-07-01 22:34:00
2     Stever 1900-07-01 16:00:00
3     Stever 1900-07-01 22:46:00
4  Forbes St 1900-07-02 18:51:00


Setup Time and Date in pandas correctly

In [19]:
# Assuming df is your DataFrame with 'Date', 'Time', and 'Location' columns

# Extract features from datetime
df['Hour'] = df['DateTime'].dt.hour
df['DayOfWeek'] = df['DateTime'].dt.dayofweek

# Encode categorical 'Location' labels
label_encoder = LabelEncoder()
df['Location_Code'] = label_encoder.fit_transform(df['Location'])

# Prepare features (X) and target (y)
X = df[['Hour', 'DayOfWeek']].values
y = df['Location_Code'].values

# Display the first few rows of the processed DataFrame to verify
print(df.head())

    Location            DateTime  Hour  DayOfWeek  Location_Code
0     Stever 1900-06-30 23:09:00    23          5              2
1     Stever 1900-07-01 22:34:00    22          6              2
2     Stever 1900-07-01 16:00:00    16          6              2
3     Stever 1900-07-01 22:46:00    22          6              2
4  Forbes St 1900-07-02 18:51:00    18          0              1


Setup OneHotEncoder

In [53]:
# One-hot encode categorical features (DayOfWeek)
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],
    remainder='passthrough'
)
X = ct.fit_transform(X)

Splitting dataset into training and testing

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Standarization via scaling

In [75]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Defining model

In [114]:
input_shape = (7,)  # Adjust according to your data

# Input layer
inputs = Input(shape=input_shape)

# Hidden layers
x = Dense(32, activation='relu')(inputs)
x = Dense(16, activation='relu')(x)

# Output layer
outputs = Dense(1, activation='sigmoid')(x)  # Example output layer for binary classification

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Model Summary

In [115]:
model.summary()

Model: "functional_40"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_21 (InputLayer)     │ (None, 7)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 32)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 801 (3.13 KB)

 Trainable params: 801 (3.13 KB)

 Non-trainable params: 0 (0.00 B)

Compile Model

In [116]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [117]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50


2024-07-05 03:18:16.276346: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 3 which is outside the valid range of [0, 1).  Label values: 2 2 2 0 0 0 2 2 3 1 2 2
2024-07-05 03:18:16.276530: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Received a label value of 3 which is outside the valid range of [0, 1).  Label values: 2 2 2 0 0 0 2 2 3 1 2 2
	 [[{{function_node __inference_one_step_on_data_2794}}{{node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/Users/alexanderokenczyc/anaconda3/envs/EnvTensor/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/Users/alexanderokenczyc/anaconda3/envs/EnvTensor/lib/python3.10/runpy.py", line 86, in _run_code

  File "/Users/alexanderokenczyc/anaconda3/envs/EnvTensor/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/alexanderokenczyc/anaconda3/envs/EnvTensor/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/alexanderokenczyc/anaconda3/envs/EnvTensor/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/alexanderokenczyc/anaconda3/envs/EnvTensor/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File